# Import Library

In [ ]:
import os  #case connect via OS
from tensorflow.keras.utils import load_img
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.utils import img_to_array
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
import graphviz
from ann_visualizer.visualize import ann_viz

In [ ]:
X_Train = []
Y_Train = []
X_Test = []
Y_Test = []
class_name = []

In [ ]:
#นำ name_folder_trainset มาใส่ใน array
directory_train = 'dataset/train'
folder_train = [folder for folder in os.listdir(directory_train) if os.path.isdir(os.path.join(directory_train, folder))]
print(folder_train)

In [ ]:
directory_test = 'dataset/test'
folder_test = [folder for folder in os.listdir(directory_test) if os.path.isdir(os.path.join(directory_test, folder))]
print(folder_test)

In [ ]:
class_name = folder_train + list(set(folder_train) - set(folder_test))
print(class_name)
print(len(class_name))

In [ ]:
for i, folder in enumerate(class_name):
  empty_array = np.zeros(len(class_name))
  empty_array[i] = 1
  files = os.listdir(directory_train+"/"+folder)
  count = 0
  for my_file in files:
    count += 1
    img = load_img(directory_train+"/"+folder+"/"+my_file)
    img = img.resize((250,250))
    img = img_to_array(img)
    Y_Train.append(empty_array)
    X_Train.append(img)
  print(folder, count)

In [ ]:
for i, folder in enumerate(class_name):
  empty_array = np.zeros(len(class_name))
  empty_array[i] = 1
  files = os.listdir(directory_test+"/"+folder)
  count = 0
  for my_file in files:
    count += 1
    img = load_img(directory_test+"/"+folder+"/"+my_file)
    img = img.resize((250,250))
    img = img_to_array(img)
    Y_Test.append(empty_array)
    X_Test.append(img)
  print(folder, count)

In [ ]:
X_Train = np.array(X_Train)
Y_Train = np.array(Y_Train)
X_Test = np.array(X_Test)
Y_Test = np.array(Y_Test)

In [ ]:
Vector = 250*250*3
X_Train = X_Train.reshape(len(Y_Train),250,250,3)
X_Test = X_Test.reshape(len(Y_Test),250,250,3)

In [ ]:
plt.rcParams['figure.figsize'] = (10,10) # change figure size to plot graph

# นำข้อมูลใน Dataset ออกมาแสดง แค่ 16 ตัวโดยจะแสดงเป็น 4 แถว 4 คอลัม
for i in range(16):
    plt.subplot(4,4,i+1)
    img_array = X_Train[i]
    plt.imshow(np.uint8(img_array))
    cl = np.argmax(Y_Train[i])
    plt.title("Class {}".format(class_name[cl]))

plt.tight_layout()

In [ ]:
# Input
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 padding='same',
                 input_shape=(250,250, 3)))
model.add(MaxPooling2D((3, 3)))

# Hidden
model.add(Dropout(0.2))
model.add(Dense(32,  activation='relu'))
# model.add(Dense(16,  activation='relu'))

# Output
model.add(Flatten())
model.add(Dense(len(class_name), activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_Train, Y_Train, batch_size=5, epochs=30, validation_data=(X_Test, Y_Test))

In [ ]:
score = model.evaluate(X_Train, Y_Train, verbose = 0)
print('Train loss : ', score[0])
print('Train accuracy : ', score[1])

In [ ]:
score1 = model.evaluate(X_Test, Y_Test, verbose = 0)
print('Test loss : ', score1[0])
print('Test accuracy : ', score1[1])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
y_pred = model.predict(X_Test)

y_test_class = np.argmax(Y_Test,axis=-1)
y_pred_class = np.argmax(y_pred,axis=-1)

print(confusion_matrix(y_test_class,y_pred_class))

In [ ]:
test_predict = tf.keras.Sequential([model,tf.keras.layers.Softmax()])
result = test_predict.predict(X_Test)
for i in range(16):
  print(class_name[int(np.argmax(result[i]))])

In [ ]:
plt.rcParams['figure.figsize'] = (9,9) # change figure size to plot graph

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(np.uint8(X_Test[i]))
    cl = np.argmax(result[i])
    plt.title("Class {}".format(class_name[cl]))
plt.tight_layout()

In [ ]:
# ann_viz(model, filename='CNN_model.gv',title=" Neural Network model from CNN")
# graph_file = graphviz.Source.from_file('CNN_model.gv')
# graph_file

# Export Model

In [ ]:
# model.save('my_model')